In [49]:
import pandas as pd
import requests
import csv
import datetime 
import locale
import time
locale.setlocale(locale.LC_ALL, 'en_US')


'en_US'

In [50]:
id_event_bolafy = pd.read_csv('id_event_bolafy.csv')
id_event_bolafy_list = list(id_event_bolafy['id_event'])

In [51]:
id_event_non_bolafy = pd.read_csv('id_event_non_bolafy.csv')
id_event_non_bolafy_list = list(id_event_non_bolafy['id_event'])

In [52]:
def get_table(table_name=''):
    """
    Returns pandas dataframe

    table_name options:

    transactions
    users
    tickets
    nft
    moflip_ticket
    """ 
    x = requests.get(f"https://api.tiketnft.com/table/export?table_name={table_name}&token=B1m4j3nie")
    text = x.text

    lines = text.splitlines()
    reader = csv.reader(lines)
    parsed_csv = list(reader)

    df = pd.DataFrame(parsed_csv[1:], columns=parsed_csv[0])
    
    return df.dropna()

In [53]:
id_event_non_bolafy_list

[74371, 75951, 75956, 75957, 75958, 75959]

In [54]:
def get_all_bolafy_trx():
    transactions_df = get_table('transactions')

    transactions_df.drop(transactions_df.index[transactions_df['invoice_payment_total'] == 'null'], inplace=True)
    transactions_df.drop(transactions_df.index[transactions_df['user_id'] == 'null'], inplace=True)


    transactions_df = transactions_df.astype(
            {
                "id_event":"int", 
                "invoice_payment_total": "int",
                "settlement_paid_timestamp": "datetime64[ns]"
            })

    transactions_df = transactions_df[transactions_df.invoice_payment_total != 0]

    bolafy_filter = ~transactions_df['id_event'].isin(id_event_non_bolafy_list)

    return transactions_df[bolafy_filter]

def get_bolafy_trx_by_event(event_ids=[]):
    all_bolafy_trx = get_all_bolafy_trx()

    if len(event_ids) == 0:
        return all_bolafy_trx


    event_filter = all_bolafy_trx['id_event'].isin(event_ids)

    return all_bolafy_trx[event_filter]

def get_bolafy_trx_by_date(all_trx, date='today', range = (1,0), format_date='%m-%d-%Y %H:%M:%S'):

    """
    date    : today, yesterday
    range   : (day,hour)

    """

    if date == 'today':
        date_explore = pd.to_datetime(datetime.datetime.today().strftime(format_date)).floor('D')
    elif date == 'yesterday':
        date_explore = pd.to_datetime(datetime.datetime.today() - datetime.timedelta(days=1)).floor('D')
        
    else: 
        date_explore = pd.to_datetime(datetime.datetime.strptime(date, format_date))

    mask = ((all_trx['settlement_paid_timestamp'] >= date_explore) & (all_trx['settlement_paid_timestamp'] < (date_explore + pd.Timedelta(days=range[0], seconds=range[1] * 3600))))
    df = all_trx.loc[mask]

    return df

def print_rupiah(num):
    return f'Rp {locale.currency(num, grouping=True)[1:]}'

def get_sales_by_event(trx=''):
    """
    trx: transaction dataframe
    """

    sales_by_id_event = trx.groupby('id_event').sum('id_event')
    all_id_event = list(sales_by_id_event.index)
    sales_by_id_event = sales_by_id_event.to_dict()

    # Load moflip ticket table and 
    moflip_ticket_table = get_table('moflip_ticket').astype({
        'event_id':'int'
    })
    mask = (moflip_ticket_table['event_id'].isin(all_id_event))
    moflip_ticket_table = moflip_ticket_table.loc[mask]

    gk = moflip_ticket_table.groupby('event_id') # Group by event id
    event_id_name = gk.first()['description'].to_dict()

    event_name_sales_dict = {}

    print(sales_by_id_event)

    for id_event in all_id_event:
       event_name_sales_dict[event_id_name[id_event]] = (sales_by_id_event['invoice_payment_total'][id_event])

    return event_name_sales_dict



def get_total_sales(trx=''):
    """
    trx: transaction dataframe

    """

    total_transaction = trx['invoice_payment_total'].sum()
    total_transaction_rupiah = print_rupiah(total_transaction)

    return total_transaction, total_transaction_rupiah

def get_num_transaction(all_trx, date='07-18-2000 00:00:00', format_date='%m-%d-%Y %H:%M:%S'):

    date_object = pd.to_datetime(datetime.datetime.strptime(date, format_date))

    mask = (all_trx['settlement_paid_timestamp'] >= date_object)
    
    return len(all_trx[mask])

# Utils function
def _get_change_percentage(a,b):

    percentage_change = 0

    try:
        percentage_change = float((b-a)/a) * 100 
    except ZeroDivisionError:
        pass

    sign = '🍀' if percentage_change > 0 else '🔻'
    
    return sign, percentage_change


In [55]:
from datetime import timedelta


def generate_today_sales_report(date='today', week_start_date='09-12-2022 00:00:00', format_date='%m-%d-%Y %H:%M:%S', event_ids=[], title=''):

    """ Improvements : 

    Harus ada generalize generate sales report from x time till x time

    Function ini sementara
    """

    cur_time = datetime.datetime.now()

    today_str = (cur_time).strftime(format_date)
    firstDayOfMonth = datetime.date(cur_time.year, cur_time.month, 1).strftime(format_date)
    
    # All Time
    all_trx = get_bolafy_trx_by_event(event_ids)

    # Today
    today_sales = get_bolafy_trx_by_date(all_trx, date)
    today_sales_amt = get_total_sales(today_sales)

    # Yesterday
    yesterday_sales = get_bolafy_trx_by_date(all_trx, 'yesterday')
    yesterday_sales_amt = get_total_sales(yesterday_sales)

    # This Week
    this_week_sales = get_bolafy_trx_by_date(all_trx, week_start_date, range=(7,0))
    this_week_sales_amt = get_total_sales(this_week_sales)

    # Last Week
    last_week_sales = get_bolafy_trx_by_date(all_trx, (datetime.datetime.strptime(week_start_date, format_date) - timedelta(days=7)).strftime(format_date), range=(7,0))
    last_week_sales_amt = get_total_sales(last_week_sales)

    # Last Month
    this_month_sales = get_bolafy_trx_by_date(all_trx, '09-01-2022 00:00:00', range=(30,0))
    this_month_sales_amt = get_total_sales(this_month_sales)    

    percentage_change = _get_change_percentage(yesterday_sales_amt[0], today_sales_amt[0])
    percentage_change_str = f"{'{0:.2f}'.format(percentage_change[1])}%"

    num_trx_all_time = get_num_transaction(all_trx)
    num_trx_month = get_num_transaction(all_trx, firstDayOfMonth)

    SALES_REPORT = f"""
    
    _🤖 Automated Message_ 

    *Bolafy Sales Report 🛒 - {title}*

    Today's Sale: {today_sales_amt[1]}

    Change from yesterday: {percentage_change[0]} {percentage_change_str}

    --- 💰 Sales --- 

    Yesterday Sales: {yesterday_sales_amt[1]}
    Last week sales: {last_week_sales_amt[1]}
    This week sales: {this_week_sales_amt[1]} (so far)
    This month sales: {this_month_sales_amt[1]} (so far)
    

    --- 🧧 Transactions ---

    Monthly Transaction: {num_trx_month}
    All Time Transaction: {num_trx_all_time}


    _Updated: {today_str}_

    """


    return str(SALES_REPORT)

In [56]:
print(generate_today_sales_report('today', week_start_date='09-19-2022 00:00:00', event_ids=[82424], title='Ismed Sofyan Campaign'))


    
    _🤖 Automated Message_ 

    *Bolafy Sales Report 🛒 - Ismed Sofyan Campaign*

    Today's Sale: Rp 0.00

    Change from yesterday: 🔻 -100.00%

    --- 💰 Sales --- 

    Yesterday Sales: Rp 289,275.00
    Last week sales: Rp 5,483,376.00
    This week sales: Rp 3,104,275.00 (so far)
    This month sales: Rp 8,587,651.00 (so far)
    

    --- 🧧 Transactions ---

    Monthly Transaction: 10
    All Time Transaction: 10


    _Updated: 09-23-2022 09:37:38_

    


In [28]:
all_trx = get_bolafy_trx_by_event(event_ids=[82424])

In [36]:
this_month_sales = get_bolafy_trx_by_date(all_trx, '09-01-2022 00:00:00', range=(30,0))
this_month_sales_amt = get_total_sales(this_month_sales) 

In [38]:
this_month_sales_amt

(8587651, 'Rp 8,587,651.00')

In [29]:
all_trx.head()

,id,user_id,id_order,evoucher_url,eticket_url,invoice_code,status_invoice_name,invoice_expired,invoice_date,voucher_status,settlement_paid_timestamp,payment_name,invoice_payment_total,id_event,status_invoice
8994,9011,12519,82424SEPXLUDWMQX,https://neo.loket.com/evoucher/36896ba6-261a-4...,https://neo.loket.com/eticket/1577409681,XLUDWMQX,PAID,2022-09-17 10:18:41,2022-09-17 10:03:21,false,2022-09-17 10:03:57,GO-PAY,1712813,82424,5
8995,9012,12519,82424SEPXLUD8BUB,https://neo.loket.com/evoucher/b43e5bea-8192-4...,https://neo.loket.com/eticket/1577409839,XLUD8BUB,PAID,2022-09-17 10:20:35,2022-09-17 10:05:21,false,2022-09-17 10:07:50,Credit / Debit Card,1449575,82424,5
8996,9013,12519,82424SEPXLUCJCQJ,https://neo.loket.com/evoucher/59163cf6-0367-4...,https://neo.loket.com/eticket/1577409963,XLUCJCQJ,PAID,2022-09-17 10:28:32,2022-09-17 10:13:18,false,2022-09-17 10:14:34,Credit / Debit Card,1449575,82424,5
8997,9014,12412,82424SEPXLUAIRCT,https://neo.loket.com/evoucher/9bf85933-b6dc-4...,https://neo.loket.com/eticket/1577410945,XLUAIRCT,PAID,2022-09-17 12:00:20,2022-09-17 10:59:55,false,2022-09-17 11:01:21,Virtual Account BCA,290000,82424,5
8999,9016,11968,82424SEPXLUGXBIH,https://neo.loket.com/evoucher/5bb1ea99-71d0-4...,https://neo.loket.com/eticket/1577413764,XLUGXBIH,PAID,2022-09-17 15:19:09,2022-09-17 15:03:50,false,2022-09-17 15:04:33,GO-PAY,144638,82424,5


In [31]:
all = get_all_bolafy_trx()

In [32]:
all.head()

,id,user_id,id_order,evoucher_url,eticket_url,invoice_code,status_invoice_name,invoice_expired,invoice_date,voucher_status,settlement_paid_timestamp,payment_name,invoice_payment_total,id_event,status_invoice
0,4,1,72755MARXOBTJWCP,https://neo.loket.com/evoucher/2d95ebde-e7ea-4...,https://neo.loket.com/eticket/1569921476,XOBTJWCP,PAID,2022-03-03 16:57:15,2022-03-03 16:42:02,false,2022-03-03 16:42:28,GO-PAY,50750,72755,null
1,5,12,72755MARXOBRVHLE,https://neo.loket.com/evoucher/c1340c59-cd0b-4...,https://neo.loket.com/eticket/1569921574,XOBRVHLE,PAID,2022-03-03 17:30:28,2022-03-03 17:15:13,false,2022-03-03 17:15:56,GO-PAY,25375,72755,null
2,8,11,72755MARXOBREHX1,https://neo.loket.com/evoucher/17492bb1-bdf2-4...,https://neo.loket.com/eticket/1569921602,XOBREHX1,PAID,2022-03-03 20:26:16,2022-03-03 17:26:05,false,2022-03-03 17:29:13,Virtual Account BCA,30000,72755,null
3,9,10,72755MARXOBNXWS8,https://neo.loket.com/evoucher/7c819c0e-108a-4...,https://neo.loket.com/eticket/1569921832,XOBNXWS8,PAID,2022-03-03 21:47:21,2022-03-03 18:46:52,false,2022-03-03 18:50:05,Virtual Account BCA,55000,72755,null
4,13,1,72753MARXOCWABZE,https://neo.loket.com/evoucher/a09c0310-1b3c-4...,https://neo.loket.com/eticket/1569931759,XOCWABZE,PAID,2022-03-05 09:38:25,2022-03-05 09:23:05,false,2022-03-05 09:23:44,GO-PAY,76125,72753,5


In [34]:
trx = get_table('transactions')

In [35]:
trx.tail(5)

,id,user_id,id_order,evoucher_url,eticket_url,invoice_code,status_invoice_name,invoice_expired,invoice_date,voucher_status,settlement_paid_timestamp,payment_name,invoice_payment_total,id_event,status_invoice
8675,8682,13778,82185SEPXLQ5O4EA,https://neo.loket.com/evoucher/c95f89c2-9980-4...,https://neo.loket.com/eticket/1577110300,XLQ5O4EA,PAID,2022-09-13 14:20:34,2022-09-13 14:05:09,false,2022-09-13 14:05:55,GO-PAY,53288,82185,5
8676,8683,16945,81840SEPXLQ6DSCZ,https://neo.loket.com/evoucher/f2e63b28-6b3d-4...,https://neo.loket.com/eticket/1577110980,XLQ6DSCZ,PAID,2022-09-13 14:41:55,2022-09-13 14:20:25,false,2022-09-13 14:28:00,GO-PAY,50750,81840,5
8677,8684,13486,81385SEPXLQ6RLXI,https://neo.loket.com/evoucher/156357ff-866e-4...,https://neo.loket.com/eticket/1577111380,XLQ6RLXI,PAID,2022-09-13 14:44:55,2022-09-13 14:29:26,false,2022-09-13 14:30:09,GO-PAY,27913,81385,5
8678,8664,4,MOFLIP-REDEEM-40M6F9SAQL,https://bolafy.com/,https://bolafy.com/,196488-X8Y1RCBL1L,PAID,2022-09-12 16:06:08,2022-09-12 16:06:08,false,2022-09-12 16:06:08,Moflip,null,81435,5
8679,8665,4,MOFLIP-REDEEM-Y9592WTOI8,https://bolafy.com/,https://bolafy.com/,196488-QQV289RYUI,PAID,2022-09-12 16:06:08,2022-09-12 16:06:08,false,2022-09-12 16:06:08,Moflip,null,81435,5


In [36]:
mev = get_table('moflip_ticket')

In [37]:
mev.tail()

,id_ticket,event_id,ticket_type,description,start_sale,end_sale,price,quantity,available_qty,available,...,fanPoint,rarity,is_popular,evoucher_show,ticket_category,required_temp,required,status,face_image_gallery,required.bms
493,199257,82225,Fadil Sausu,Koleksi Digital Best XI: BRI Liga 1 2022/2023,Tue Sep 13 2022 00:00:00 GMT+0000 (Coordinated...,Wed Sep 21 2022 00:00:00 GMT+0000 (Coordinated...,75000,100,100,true,...,60,Rare,false,false,nft,null,null,null,null,null
494,199258,82225,Beckham Putra,Koleksi Digital Best XI: BRI Liga 1 2022/2023,Tue Sep 13 2022 00:00:00 GMT+0000 (Coordinated...,Wed Sep 21 2022 00:00:00 GMT+0000 (Coordinated...,75000,100,100,true,...,60,Rare,false,false,nft,null,null,null,null,null
495,199259,82225,Hugo Gomez,Koleksi Digital Best XI: BRI Liga 1 2022/2023,Tue Sep 13 2022 00:00:00 GMT+0000 (Coordinated...,Wed Sep 21 2022 00:00:00 GMT+0000 (Coordinated...,75000,100,100,true,...,60,Rare,false,false,nft,null,null,null,null,null
496,199260,82225,Yakob Sayuri,Koleksi Digital Best XI: BRI Liga 1 2022/2023,Tue Sep 13 2022 00:00:00 GMT+0000 (Coordinated...,Wed Sep 21 2022 00:00:00 GMT+0000 (Coordinated...,75000,100,100,true,...,60,Rare,false,false,nft,null,null,null,null,null
497,199261,82225,Riyan Ardiansyah,Koleksi Digital Best XI: BRI Liga 1 2022/2023,Tue Sep 13 2022 00:00:00 GMT+0000 (Coordinated...,Wed Sep 21 2022 00:00:00 GMT+0000 (Coordinated...,75000,100,100,true,...,60,Rare,false,false,nft,null,null,null,null,null


In [47]:
sales_by_id_event = all_trx.groupby('id_event')



In [50]:
sales_by_id_event = weekly_sales.groupby('id_event').sum('id_event')
all_id_event = list(sales_by_id_event.index)
sales_by_id_event = sales_by_id_event.to_dict()

# Load moflip ticket table and 
moflip_ticket_table = get_table('moflip_ticket').astype({
    'event_id':'int'
})
mask = (moflip_ticket_table['event_id'].isin(all_id_event))
moflip_ticket_table = moflip_ticket_table.loc[mask]

gk = moflip_ticket_table.groupby('event_id') # Group by event id
event_id_name = gk.first()['description'].to_dict()

event_name_sales_dict = {}

print(sales_by_id_event)

for id_event in all_id_event:
    event_name_sales_dict[event_id_name[id_event]] = (sales_by_id_event['invoice_payment_total'][id_event])

{'invoice_payment_total': {79594: 77100, 81385: 27913, 81840: 50750, 82185: 53288}}


In [51]:
event_name_sales_dict

{'Persikabo 1973 vs PERSIJA Jakarta': 77100,
 'Momen Eksklusif Borneo FC: Kilas Balik Week 1 - 7': 27913,
 'PERSIJA Jakarta vs Bhayangkara FC': 50750,
 'PS Barito Putera vs PERSIJA Jakarta': 53288}

In [57]:
mev.loc[mev['event_id'] == '81435']

,id_ticket,event_id,ticket_type,description,start_sale,end_sale,price,quantity,available_qty,available,...,fanPoint,rarity,is_popular,evoucher_show,ticket_category,required_temp,required,status,face_image_gallery,required.bms
425,196482,81435,Andritany Ardhiyasa,Koleksi Digital Best XI: BRI Liga 1 2022/2023,Fri Aug 26 2022 00:00:00 GMT+0000 (Coordinated...,Sat Sep 03 2022 00:00:00 GMT+0000 (Coordinated...,75000,100,100,true,...,60,Rare,false,false,nft,null,null,null,null,null
427,196483,81435,Putu Gede Juniantara,Koleksi Digital Best XI: BRI Liga 1 2022/2023,Fri Aug 26 2022 00:00:00 GMT+0000 (Coordinated...,Sat Sep 03 2022 00:00:00 GMT+0000 (Coordinated...,75000,100,100,true,...,60,Rare,false,false,nft,null,null,null,null,null
428,196491,81435,Matheus Pato,Koleksi Digital Best XI: BRI Liga 1 2022/2023,Fri Aug 26 2022 00:00:00 GMT+0000 (Coordinated...,Sat Sep 03 2022 00:00:00 GMT+0000 (Coordinated...,75000,100,100,true,...,60,Rare,false,false,nft,null,null,null,null,null
429,196459,81435,Privat Mbarga,Koleksi Digital Best XI: BRI Liga 1 2022/2023,Fri Aug 26 2022 00:00:00 GMT+0000 (Coordinated...,Sat Sep 03 2022 00:00:00 GMT+0000 (Coordinated...,249000,10,10,true,...,200,Super Rare,false,false,nft,null,null,null,null,null
444,196484,81435,Rizky Ridho,Koleksi Digital Best XI: BRI Liga 1 2022/2023,Fri Aug 26 2022 00:00:00 GMT+0000 (Coordinated...,Sat Sep 03 2022 00:00:00 GMT+0000 (Coordinated...,75000,100,100,true,...,60,Rare,false,false,nft,null,null,null,null,null
445,196485,81435,Tallysson Duarte,Koleksi Digital Best XI: BRI Liga 1 2022/2023,Fri Aug 26 2022 00:00:00 GMT+0000 (Coordinated...,Sat Sep 03 2022 00:00:00 GMT+0000 (Coordinated...,75000,100,100,true,...,60,Rare,false,false,nft,null,null,null,null,null
446,196486,81435,Ricky Fajrin,Koleksi Digital Best XI: BRI Liga 1 2022/2023,Fri Aug 26 2022 00:00:00 GMT+0000 (Coordinated...,Sat Sep 03 2022 00:00:00 GMT+0000 (Coordinated...,75000,100,100,true,...,60,Rare,false,false,nft,null,null,null,null,null
447,196487,81435,Jayus Hariono,Koleksi Digital Best XI: BRI Liga 1 2022/2023,Fri Aug 26 2022 00:00:00 GMT+0000 (Coordinated...,Sat Sep 03 2022 00:00:00 GMT+0000 (Coordinated...,75000,100,100,true,...,60,Rare,false,false,nft,null,null,null,null,null
448,196488,81435,Marselino Ferdinan,Koleksi Digital Best XI: BRI Liga 1 2022/2023,Fri Aug 26 2022 00:00:00 GMT+0000 (Coordinated...,Sat Sep 03 2022 00:00:00 GMT+0000 (Coordinated...,75000,100,100,true,...,60,Rare,false,false,nft,null,null,null,null,null
449,196489,81435,Alexis Messidoro,Koleksi Digital Best XI: BRI Liga 1 2022/2023,Fri Aug 26 2022 00:00:00 GMT+0000 (Coordinated...,Sat Sep 03 2022 00:00:00 GMT+0000 (Coordinated...,75000,100,100,true,...,60,Rare,false,false,nft,null,null,null,null,null


In [58]:
t = get_table('tickets')

In [60]:
t.tail(5)

,id,id_ticket,user_id,txns_id,ticket_quantity,ticket_price,ticket_type,nft_image,payment_name,status_invoice_name,id_event,organization_id,barcode
9086,9093,199116,13778,8682,1,50000,The Goal Machine: Michael Krmencik,,GO-PAY,PAID,82185,null,8218536436317221
9087,9094,198003,16945,8683,1,50000,Konsisten: Empat Kemenangan Beruntun,,GO-PAY,PAID,81840,null,8184002181167869
9088,9095,196301,13486,8684,1,25000,Tendangan Bebas Mematikan : Matheus Pato,,GO-PAY,PAID,81385,null,8138557693039876
9089,9074,196488,4,8664,1,0,Marselino Ferdinan,,Moflip,PAID,81435,null,57590008
9090,9075,196488,4,8665,1,0,Marselino Ferdinan,,Moflip,PAID,81435,null,75430197


In [5]:
nft = get_table('nft')

In [12]:
nft.sample(3)

,id,ticket_id,ticket_counter,preview,nft_images,contract_address,ipfs_hash,open_sea_link,user_id,original_owner_id,...,title,description,moflip_ticket_origin,barcode,evoucher_url,rarity,owner_name,owner_company,redeem_code,eticket_url
13953,14234,9214,1,null,null,null,null,null,null,null,...,,,null,93907435,https://bolafy.com/,Rare,null,null,MC93907435,null
12608,12794,7855,1,null,null,0x2953399124f0cbb46d2cbacd8a89cf0599974963,null,https://opensea.io/assets/matic/0x2953399124f0...,11867,11867,...,PERSIJA Jakarta vs PERSIS Solo #057,Koleksi Tiket NFT - PERSIJA Jakarta vs PERSIS ...,null,7959403435339866,https://neo.loket.com/evoucher/3b008c8b-4e0f-4...,Limited,null,null,null,null
6178,4685,2304,1,null,null,0x6143db544535e6cdc78669ca4499b016dab0ff55,null,https://opensea.io/assets/matic/0x6143db544535...,5360,5360,...,Baladewa Festival #1428,"Selamat, Anda adalah pemegang Baladewa 30 Fest...",null,7595107745298604,https://neo.loket.com/evoucher/9aac33a3-400f-4...,Common,null,null,null,null
